<a href="https://colab.research.google.com/github/rehmanaly0051/CGAN-Model-Training/blob/main/Conditional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Dataset Preprocessing

In [19]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

### Defining Dataset class

In [20]:
class CubiCasa5kDataset(Dataset):
    def __init__(self, data_list, root_dir, category_list, transforms=None):
        if not isinstance(category_list, list):
            raise TypeError(f"Expected category_list to be a list, got {type(category_list)} instead.")

        self.data_list = data_list if data_list is not None else []
        self.root_dir = root_dir
        self.category_list = category_list
        self.transforms = transforms

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        image_id = self.data_list[idx]
        # Remove the directory path from image_id if it exists
        image_id = os.path.basename(image_id)

        for category in self.category_list:
            img_path = os.path.join(self.root_dir, category, f"{image_id}_original.png")
            label_path = os.path.join(self.root_dir, category, f"{image_id}_scaled.png")

            if os.path.exists(img_path) and os.path.exists(label_path):
                image = Image.open(img_path).convert("RGB")
                label = Image.open(label_path).convert("RGB")

                if self.transforms:
                    image = self.transforms(image)
                    label = self.transforms(label)

                return image, label

        raise FileNotFoundError(f"Image {image_id} not found in any category")

### Dataset path

In [21]:
dataset_path = "/content/drive/MyDrive/cubicasa5k"
categories = ["high_quality_architectural", "high_quality", "colorful"]  # Ensure it's a list

# Define transforms
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

### Loading Train, Test and Val labels

In [22]:
import os

def load_labels(file_path):
    if not os.path.exists(file_path):
        print(f"Warning: File not found: {file_path}")
        return []  # Return an empty list if the file doesn't exist

    with open(file_path, "r") as f:
        lines = f.readlines()

    labels = [line.strip() for line in lines if line.strip()]  # Remove empty lines

    if not labels:
        print(f"Warning: File is empty: {file_path}")

    return labels

# Example usage
train_data = load_labels(os.path.join(dataset_path, "train.txt"))
test_data = load_labels(os.path.join(dataset_path, "test.txt"))
val_data = load_labels(os.path.join(dataset_path, "val.txt"))

### Defining transfomations

### Creating dataset

In [23]:
dataset_train = CubiCasa5kDataset(train_data, dataset_path, categories, transforms)
dataset_test = CubiCasa5kDataset(test_data, dataset_path, categories, transforms)
dataset_val = CubiCasa5kDataset(val_data, dataset_path, categories, transforms)

### Creating dataloaders

In [24]:
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False)

### Displaying dataset size

In [25]:
print(f'Train Samples : {len(dataset_train)}',
      f'Test Samples : {len(dataset_test)}',
      f'Val Samples : {len(dataset_val)}')

Train Samples : 4200 Test Samples : 400 Val Samples : 400


## CGAN floorplan Generation

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torch.utils.data import DataLoader

### Defining Generator class

In [27]:
class Generator(nn.Module):
    def __init__(self, latent_dim, condition_dim, output_channels):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_channels),
            nn.Tanh()
        )

    def forward(self, z, condition):
        x = torch.cat((z, condition), dim=1)
        return self.model(x)

### Defining Discriminator class

In [28]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, condition_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + condition_dim, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img, condition):
        x = torch.cat((img, condition), dim=1)
        return self.model(x)

### Setting parameters

In [29]:
condition_dim = 2 # Assuming user inputs (width, height)
latent_dim = 100
image_dim = 256 * 256 * 3
batch_size = 16
num_epochs = 100
learning_rate = 0.0002

### Initializing Models

In [30]:
generator = Generator(latent_dim, condition_dim, image_dim)
discriminator = Discriminator(image_dim, condition_dim)

gan_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

### Training Loop

In [31]:
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)

dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

# Create output folder
output_folder = "output_images"
os.makedirs(output_folder, exist_ok=True)

for epoch in range(epochs):
    for i, (real_imgs, conditions) in enumerate(dataloader):
        real_imgs = real_imgs.view(batch_size, -1).to(device)
        conditions = conditions.to(device)  # User-defined dimensions

        # Train Discriminator
        optimizer_D.zero_grad()
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = generator(z, conditions).detach()

        real_loss = gan_loss(discriminator(real_imgs, conditions), torch.ones(batch_size, 1).to(device))
        fake_loss = gan_loss(discriminator(fake_imgs, conditions), torch.zeros(batch_size, 1).to(device))
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        fake_imgs = generator(z, conditions)
        g_loss = gan_loss(discriminator(fake_imgs, conditions), torch.ones(batch_size, 1).to(device))
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch {epoch}/{epochs} | D Loss: {d_loss.item()} | G Loss: {g_loss.item()}")

    if epoch % 10 == 0:
        save_path = os.path.join(output_folder, f"generated_epoch_{epoch}.png")
        vutils.save_image(fake_imgs.view(batch_size, 3, 256, 256), save_path, normalize=True)

FileNotFoundError: Image  not found in any category